In [1]:
import openai
import yfinance as yf
import json

In [2]:
def get_stock_price(symbol):
    stock = yf.Ticker(symbol)
    price = stock.info['currentPrice']
    return price

In [3]:
def get_latest_company_news(symbol):
    stock = yf.Ticker(symbol)
    news = stock.news
    # 최신 뉴스 3개 리스트에 저장하기
    news_list = []
    num =1
    for item in news[:3]:
        news_list.append(f"{num}: title : "+item['title']+", publisher:"+item['publisher']+", link :"+item['link'])
        num+=1
    return news_list

In [4]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The ticker symbol of the stock"
                    }
                },
                "required": ["symbol"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_latest_company_news",
            "description": "Fetches the latest news articles related to a specified company",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The ticker symbol of the stock"
                    }
                },
                "required": ["symbol"]
            }
        }
    }
]

In [5]:
client = openai.OpenAI(api_key = "여기에 API 키를 넣어주세요")

In [6]:
instruction = '''
너는 주식 분석 전문가야

규칙:
1. 사용자가 회사 또는 종목에 대해 물어보면, function을 활용해서 회사의 주식 가격과 최신 뉴스를 답해줘.
2. 모든 답변은 한국어로 해줘.
'''

# Add the file to the assistant
assistant = client.beta.assistants.create(
    name = "Company Stock Analyst",
    instructions=instruction,
    model="gpt-3.5-turbo",
    tools=tools_list,
)

In [7]:
thread = client.beta.threads.create()

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="테슬라의 최근 주가와 최신 뉴스에 대해 알려줘"
)

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [10]:
import time

while run.status not in ["completed", "failed", "requires_action"]:
    run = client.beta.threads.runs.retrieve(
        thread_id= thread.id,
        run_id= run.id
    )
    print(run.status)
    time.sleep(5)

print(run.status)

requires_action
requires_action


In [11]:
tools_to_call = run.required_action.submit_tool_outputs.tool_calls
print("필요 호출 개수: ", len(tools_to_call))
print(tools_to_call)

필요 호출 개수:  2
[RequiredActionFunctionToolCall(id='call_pYwZ8cBcozpeHrfWOw1jrhWF', function=Function(arguments='{"symbol": "TSLA"}', name='get_stock_price'), type='function'), RequiredActionFunctionToolCall(id='call_Iqyu7BqeSnjAhG6BRomlS0HK', function=Function(arguments='{"symbol": "TSLA"}', name='get_latest_company_news'), type='function')]


In [12]:
print(tools_to_call[0].function.name)
print(tools_to_call[0].function.arguments)
print(tools_to_call[1].function.name)
print(tools_to_call[1].function.arguments)

get_stock_price
{"symbol": "TSLA"}
get_latest_company_news
{"symbol": "TSLA"}


In [13]:
tools_output_array = []
for each_tool in tools_to_call:
    tool_call_id = each_tool.id
    function_name = each_tool.function.name
    function_arg = each_tool.function.arguments
    # Json 포매팅
    function_arg = json.loads(each_tool.function.arguments)
    if (function_name == 'get_stock_price'):
        ## Custom 함수 생성 ##
        output=get_stock_price(function_arg["symbol"])
    if (function_name == 'get_latest_company_news'):
        ## Custom 함수 생성 ##
        output=get_latest_company_news(function_arg["symbol"])
    tools_output_array.append({"tool_call_id": tool_call_id, "output":
json.dumps(output)})

print(tools_output_array)

[{'tool_call_id': 'call_pYwZ8cBcozpeHrfWOw1jrhWF', 'output': '227.2'}, {'tool_call_id': 'call_Iqyu7BqeSnjAhG6BRomlS0HK', 'output': '["1: title : Dow Jones Futures Rise After Stock Market Falls Following Big Fed Rate Cut; What To Do Now, publisher:Investor\'s Business Daily, link :https://finance.yahoo.com/m/f96bcb1a-58f3-3e8e-a98a-e014e296338b/dow-jones-futures-rise-after.html", "2: title : How interest rate cuts will impact vehicle sales, auto loans, publisher:Yahoo Finance Video, link :https://finance.yahoo.com/video/interest-rate-cuts-impact-vehicle-205354559.html", "3: title : These Stocks Moved the Most Today: Intuitive Machines, Super Micro, Boeing, VF Corp., ResMed, General Mills, and More, publisher:Barrons.com, link :https://finance.yahoo.com/m/aaa3b566-4045-387a-af16-30ef061da446/these-stocks-moved-the-most.html"]'}]


In [14]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run.id,
    tool_outputs=tools_output_array
)

In [15]:
while run.status not in ["completed", "failed", "requires_action"]:
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
    print(run.status)
    time.sleep(5)

completed


In [16]:
# 메시지 리스트 가져오기
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# 메시지 출력
for each in messages:
    print(each.role + ": "+each.content[0].text.value)
    print("=========")

assistant: 테슬라(TSLA)의 최근 주가는 227.2달러입니다.

최신 뉴스:
1. 제목: 다우 존스 선물이 대형 연준 금리 인하 이후 주가 하락 후 상승; 지금 어떻게 해야 할까?, 발행사: Investor's Business Daily, 링크: [기사 링크](https://finance.yahoo.com/m/f96bcb1a-58f3-3e8e-a98a-e014e296338b/dow-jones-futures-rise-after.html)
2. 제목: 금리 인하가 차량 판매, 자동차 대출에 미치는 영향, 발행사: Yahoo Finance Video, 링크: [기사 링크](https://finance.yahoo.com/video/interest-rate-cuts-impact-vehicle-205354559.html)
3. 제목: 오늘 주식 시장에서 가장 많이 움직인 종목: Intuitive Machines, Super Micro, Boeing, VF Corp., ResMed, General Mills 등, 발행사: Barrons.com, 링크: [기사 링크](https://finance.yahoo.com/m/aaa3b566-4045-387a-af16-30ef061da446/these-stocks-moved-the-most.html)
user: 테슬라의 최근 주가와 최신 뉴스에 대해 알려줘
